# Lecture 4 -  K-fold cross validation

## Load the CCHS data from epi7913A package

In [1]:
library(magrittr)

# obtain a 10% random subsample of the cchs data
full_data <- epi7913A::cchs %>% dplyr::slice_sample(prop=0.1)
head(full_data)

,age,sex,CANHEARTbin,householdsize,education,maritalstatus,immigration,houseincome
,<fct>,<fct>,<int>,<fct>,<fct>,<fct>,<int>,<fct>
1,6,1,0,2,4,3,0,3
2,3,2,1,3,4,1,0,1
3,4,2,1,4,4,3,1,1
4,7,1,1,2,4,3,0,1
5,7,1,1,2,4,3,0,2
6,4,2,0,2,4,1,0,2


## Define the outcome variable

In [2]:
# define the outcome variable
voutcome<-"CANHEARTbin"

## Calculate the mean performance of K-fold cross validation:
#### - Create K folds
#### - For each fold i, designate data in the remaining K-1 folds for training and those in fold i for testing
#### - Perform model optimization on the training data
#### - Obtain predictions on the test fold using the optimal model found
#### - Calculate the logloss of predictions made

In [3]:
# define a function to calculate the mean performance of K-fold cross validation
#    - create K folds
#    - For each fold i, designate data in the remaining K-1 folds for training and those in fold i for testing
#.   - Perform model optimization on the training data
#.   - Obtain predictions on the test fold using the optimal model found
#    - Calculate the logloss of predictions made
ll.mean<-mean(sapply(caret::createFolds(full_data[, voutcome], k=5), function(x){

  # split the train and test data for each fold
  testInds <- x
  trnInds <- setdiff(1:nrow(full_data), testInds)
  train_data <- full_data[trnInds,]
  test_data <- full_data[testInds,]

  # train the optimal model using 5 iterations of nested cross-validation
  best_model<-sdgm::cart.bestmodel.bin(train_data, voutcome, n_iter=5)

  # obtain predicitons of the test fold
  preds<-predict(best_model, test_data)

  # calculate the logloss for the fold
  if (!is.null(preds))
    test_ll<- MLmetrics::LogLoss(preds, test_data[,voutcome] )
  else {
    test_ll<-NA
    print("Logloss calculation failed")
  }
}), na.rm=T)

# Print the mean logloss of the results
cat("\nThe mean logloss obtained from nested cross-validation is:",ll.mean,"\n")


 Best Parameters Found: 
Round = 24	minsplit = 7.0000	minbucket = 10.0000	cp = 0.01711718	maxdepth = 13.0000	Value = -0.6301812 

 Best Parameters Found: 
Round = 17	minsplit = 17.0000	minbucket = 14.0000	cp = 0.01415401	maxdepth = 7.0000	Value = -0.6182809 

 Best Parameters Found: 
Round = 4	minsplit = 9.0000	minbucket = 10.0000	cp = 0.01819298	maxdepth = 10.0000	Value = -0.6096636 

 Best Parameters Found: 
Round = 1	minsplit = 16.0000	minbucket = 14.0000	cp = 0.01723597	maxdepth = 10.0000	Value = -0.6040377 

 Best Parameters Found: 
Round = 1	minsplit = 13.0000	minbucket = 7.0000	cp = 0.02010101	maxdepth = 9.0000	Value = -0.6086509 

The mean logloss obtained from nested cross-validation is: 0.6202667 
